In [1]:
from grpc import insecure_channel
from ml_metadata.proto import metadata_store_pb2
from ml_metadata.proto import metadata_store_service_pb2
from ml_metadata.proto import metadata_store_service_pb2_grpc
channel = insecure_channel('metadata-grpc-service.kubeflow:8080')
stub = metadata_store_service_pb2_grpc.MetadataStoreServiceStub(channel)

In [2]:
# Create ArtifactTypes, e.g., Data and Model
data_type = metadata_store_pb2.ArtifactType()
data_type.name = "DataSet"
data_type.properties["day"] = metadata_store_pb2.INT
data_type.properties["split"] = metadata_store_pb2.STRING
request = metadata_store_service_pb2.PutArtifactTypeRequest()
request.all_fields_match = True
request.artifact_type.CopyFrom(data_type)
data_type_id = stub.PutArtifactType(request).type_id

model_type = metadata_store_pb2.ArtifactType()
model_type.name = "SavedModel"
model_type.properties["version"] = metadata_store_pb2.INT
model_type.properties["name"] = metadata_store_pb2.STRING
request.artifact_type.CopyFrom(model_type)
model_type_id = stub.PutArtifactType(request).type_id

In [3]:
# Create ExecutionType, e.g., Trainer
trainer_type = metadata_store_pb2.ExecutionType()
trainer_type.name = "Trainer"
trainer_type.properties["state"] = metadata_store_pb2.STRING
request = metadata_store_service_pb2.PutExecutionTypeRequest()
request.all_fields_match = True
request.execution_type.CopyFrom(trainer_type)
trainer_type_id = stub.PutExecutionType(request).type_id

In [4]:
# Declare input artifact of type DataSet
data_artifact = metadata_store_pb2.Artifact()
data_artifact.uri = 'path/to/data'
data_artifact.properties["day"].int_value = 1
data_artifact.properties["split"].string_value = 'train'
data_artifact.type_id = data_type_id
request = metadata_store_service_pb2.PutArtifactsRequest()
request.artifacts.add().CopyFrom(data_artifact)
data_artifact_id = stub.PutArtifacts(request).artifact_ids[0]

In [5]:
# Register the Execution of a Trainer run
trainer_run = metadata_store_pb2.Execution()
trainer_run.type_id = trainer_type_id
trainer_run.properties["state"].string_value = "RUNNING"
request = metadata_store_service_pb2.PutExecutionsRequest()
request.executions.add().CopyFrom(trainer_run)
run_id = stub.PutExecutions(request).execution_ids[0]

In [6]:
# Declare the input event
input_event = metadata_store_pb2.Event()
input_event.artifact_id = data_artifact_id
input_event.execution_id = run_id
input_event.type = metadata_store_pb2.Event.DECLARED_INPUT

# Submit input event to the Metadata Store
request = metadata_store_service_pb2.PutEventsRequest()
request.events.add().CopyFrom(input_event)
stub.PutEvents(request)

In [7]:
# Declare output artifact of type SavedModel
model_artifact = metadata_store_pb2.Artifact()
model_artifact.uri = 'path/to/model/file'
model_artifact.properties["version"].int_value = 1
model_artifact.properties["name"].string_value = 'MNIST-v1'
model_artifact.type_id = model_type_id
request = metadata_store_service_pb2.PutArtifactsRequest()
request.artifacts.add().CopyFrom(model_artifact)
model_artifact_id = stub.PutArtifacts(request).artifact_ids[0]

In [8]:
# Declare the output event
output_event = metadata_store_pb2.Event()
output_event.artifact_id = model_artifact_id
output_event.execution_id = run_id
output_event.type = metadata_store_pb2.Event.DECLARED_OUTPUT

# Submit output event to the Metadata Store
request = metadata_store_service_pb2.PutEventsRequest()
request.events.add().CopyFrom(output_event)
stub.PutEvents(request)

In [9]:
trainer_run.id = run_id
trainer_run.properties["state"].string_value = "COMPLETED"
request = metadata_store_service_pb2.PutExecutionsRequest()
request.executions.add().CopyFrom(trainer_run)
stub.PutExecutions(request)

execution_ids: 33

In [10]:
# Similarly, create a ContextType, e.g., Experiment with a `note` property
experiment_type = metadata_store_pb2.ContextType()
experiment_type.name = "Experiment"
experiment_type.properties["note"] = metadata_store_pb2.STRING
request = metadata_store_service_pb2.PutContextTypeRequest()
request.all_fields_match = True
request.context_type.CopyFrom(experiment_type)
experiment_type_id = stub.PutContextType(request).type_id


# Group the model and the trainer run to an experiment.
my_experiment = metadata_store_pb2.Context()
my_experiment.type_id = experiment_type_id
# Give the experiment a name
my_experiment.name = "exp2-grpc"
my_experiment.properties["note"].string_value = "My second experiment."
request = metadata_store_service_pb2.PutContextsRequest()
request.contexts.add().CopyFrom(my_experiment)
experiment_id = stub.PutContexts(request).context_ids[0]


attribution = metadata_store_pb2.Attribution()
attribution.artifact_id = model_artifact_id
attribution.context_id = experiment_id

association = metadata_store_pb2.Association()
association.execution_id = run_id
association.context_id = experiment_id

request = metadata_store_service_pb2.PutAttributionsAndAssociationsRequest()
request.attributions.add().CopyFrom(attribution)
request.associations.add().CopyFrom(association)
stub.PutAttributionsAndAssociations(request)